## Steerable MinMaxScaler

In [1]:
import pandas as pd
import numpy as np
import tempfile as tf
import os
from progressivis import Scheduler, Print
from progressivis.io.api import SimpleCSVLoader, Variable
from progressivis.stats.api import Histogram2D, Min, Max
from progressivis.datasets import get_dataset
from progressivis.vis import Heatmap
from progressivis.table.constant import ConstDict
from progressivis.utils.psdict import PDict
from progressivis.stats.scaling import MinMaxScaler
from ipyprogressivis.widgets.iscaler import IScalerIn, IScalerOut
from progressivis.datasets import get_dataset

s = Scheduler.default = Scheduler()
cols = ['A', 'B']
csv = SimpleCSVLoader(get_dataset('bigfile_multiscale'), usecols=cols, throttle=100)
dvar = Variable({'delta': -5, 'ignore_max':10})
sc = MinMaxScaler(reset_threshold=10_000)
sc.create_dependent_modules(csv)
sc.control = dvar
sc.input.control = dvar.output.result
pr=Print(proc=lambda x: None, scheduler=s)
pr.input[0] = sc.output.info
histogram2d = Histogram2D('A', # compute a 2d histogram
                          'B', 
                          xbins=256, ybins=256)
histogram2d.input.table = sc.output.result
cmin = ConstDict(pdict=PDict({'A': 0.0, 'B':0.0}))
cmax = ConstDict(pdict=PDict({'A': 1.0, 'B':1.0}))
histogram2d.input.min = cmin.output.result
histogram2d.input.max = cmax.output.result
heatmap=Heatmap() # compute the Heatmap
heatmap.input.array = histogram2d.output.result

**NB:** the results will appear below after running all cells :

In [2]:
import ipywidgets as ipw
from IPython.display import display
wg = None
async def _after_run(m, run_number):
    global wg
    img = m.get_image_bin()
    if img is None:
        return
    if wg is None:
        wg = ipw.Image(value=img, width=512, height=512)
        display(wg)
    else:
        wg.value = img
heatmap.on_after_run(_after_run)
isc = IScalerIn(sc)
display(isc)
isco = IScalerOut(sc)
display(isco)
s.task_start();

IScalerIn(children=(Label(value='Reset threshold:'), IntSlider(value=1000, continuous_update=False, max=100000…

IScalerOut(children=(GridBox(children=(Label(value='Clipped:'), Label(value=''), Label(value='Ignored:'), Labe…

Starting scheduler
# Scheduler added module(s): ['const_dict_1', 'const_dict_2', 'heatmap_1', 'histogram2_d_1', 'max_1', 'min_1', 'min_max_scaler_1', 'print_1', 'simple_csv_loader_1', 'variable_1']


Image(value=b'\x89PNG\r\n\x1a\n\x00\x00\x00\rIHDR\x00\x00\x01\x00\x00\x00\x01\x00\x10\x00\x00\x00\x00)\x89+\xf…